# Python Concurrency and Networking
This notebook covers threading, multiprocessing, async/await, sockets, and HTTP requests with real-life use cases, best practices, and code examples.

## 1. Threading
**Definition:** Threading allows concurrent execution of code, useful for I/O-bound tasks.

**Syntax and Example:**

### Importing Threading and Time Modules

**Introduction:**
To use threading in Python, you need to import the `threading` and `time` modules.

**Real-life use case:**
These modules are used for running concurrent tasks and simulating delays in code.

**What the code does:**
The next code cell imports the required modules for threading examples.

In [ ]:
import threading
import time

### Basic Threading Example

**Introduction:**
A thread allows you to run a function concurrently with the main program.

**Real-life use case:**
Downloading files in the background while the main program continues.

**What the code does:**
The next code cell defines a function that prints numbers with a delay, and shows how to run it in a separate thread.

In [ ]:
def print_numbers():
    """Print numbers with a delay to simulate work."""
    for i in range(3):
        print(f'Thread: {i}')
        time.sleep(1)

# Create and start a thread
thread = threading.Thread(target=print_numbers)
thread.start()
print("Main thread continues while child thread runs")
thread.join()
print("Thread completed")

### Running Multiple Threads in Parallel

**Introduction:**
You can start several threads to perform tasks in parallel, which is useful for handling multiple I/O operations at once.

**Real-life use case:**
Processing multiple files or network requests simultaneously.

**What the code does:**
The next code cell defines a worker function and starts multiple threads to run it.

In [ ]:
def worker(name):
    """Worker function that reports its name and timestamps."""
    print(f'Worker {name} starting')
    time.sleep(2)
    print(f'Worker {name} finished')

threads = []
for i in range(3):
    t = threading.Thread(target=worker, args=(f'#{i}',))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
print("All workers completed")

### Running a Background Task with a Daemon Thread

**Introduction:**
Daemon threads run in the background and automatically exit when the main program finishes.

**Real-life use case:**
Background monitoring or periodic tasks that should not block program exit.

**What the code does:**
The next code cell creates a daemon thread for a background task while the main thread continues its work.

In [ ]:
def background_task():
    for i in range(5):
        print(f"Background task: step {i}")
        time.sleep(0.5)

background_thread = threading.Thread(target=background_task)
background_thread.daemon = True
background_thread.start()
for i in range(3):
    print(f"Main thread: step {i}")
    time.sleep(0.7)
print("Main thread finished (background thread may not have completed)")

### Thread Synchronization with Lock

**Introduction:**
When multiple threads access shared data, you need synchronization to prevent race conditions.

**Real-life use case:**
Safely updating a shared counter from multiple threads.

**What the code does:**
The next code cell demonstrates using a Lock to synchronize access to a shared variable.

In [ ]:
counter = 0
count_lock = threading.Lock()

def increment_counter():
    global counter
    for _ in range(100000):
        with count_lock:
            counter += 1
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)
thread1.start()
thread2.start()
thread1.join()
thread2.join()
print(f"Final counter value with locks: {counter}")

### Producer-Consumer Pattern with Queue

**Introduction:**
The producer-consumer pattern uses a queue to safely pass data between threads.

**Real-life use case:**
A web crawler where one thread fetches URLs (producer) and another processes the content (consumer).

**What the code does:**
The next code cell demonstrates a producer thread putting items in a queue and a consumer thread processing them.

In [ ]:
import queue

task_queue = queue.Queue()

def producer():
    for i in range(5):
        item = f"Item-{i}"
        task_queue.put(item)
        print(f"Produced {item}")
        time.sleep(0.5)
    task_queue.put(None)

def consumer():
    while True:
        item = task_queue.get()
        if item is None:
            break
        print(f"Consumed {item}")
        task_queue.task_done()
        time.sleep(1)
prod_thread = threading.Thread(target=producer)
cons_thread = threading.Thread(target=consumer)
prod_thread.start()
cons_thread.start()
prod_thread.join()
cons_thread.join()
print("Producer-Consumer demonstration completed")

### Note on Python's Global Interpreter Lock (GIL)

**Introduction:**
The Global Interpreter Lock (GIL) affects how threads execute Python code.

**Real-life use case:**
Understanding the GIL helps you choose between threading and multiprocessing for your application.

**What the code does:**
The next code cell explains the GIL and when to use threading vs. multiprocessing.

In [ ]:
print("The GIL prevents multiple native threads from executing Python bytecode at once.")
print("This means threading is most useful for I/O-bound tasks rather than CPU-bound tasks.")
print("For CPU-bound tasks, consider multiprocessing instead.")

**Output:**
Thread: 0
Thread: 1
Thread: 2

**Real-life use case:** Downloading multiple files at the same time.

**Common mistakes:** Not using locks for shared data, leading to race conditions.

**Best practices:** Use threading for I/O-bound tasks and use locks for shared resources.

## 2. Multiprocessing
**Definition:** Multiprocessing runs code in separate processes, ideal for CPU-bound tasks.

**Syntax and Example:**

In [ ]:
import multiprocessing
import time
import os
import numpy as np

# Basic multiprocessing example with Pool
print("Basic multiprocessing Pool example:")

def square(x):
    """Simple CPU-bound function that squares a number"""
    # Display process ID to show work happens in different processes
    process_id = os.getpid()
    print(f"Process {process_id}: Computing square of {x}")
    # Simulate complex computation
    time.sleep(0.5)
    return x * x

# Using a process pool to execute tasks in parallel
if __name__ == '__main__':  # Required for Windows compatibility
    # Create a pool with 2 worker processes
    with multiprocessing.Pool(processes=2) as pool:
        # Map the square function to a list of inputs
        results = pool.map(square, [1, 2, 3, 4])
        print(f"Results: {results}")

# Using Process class directly
print("\nUsing the Process class directly:")

def worker_function(name):
    """Function to be executed in a separate process"""
    process_id = os.getpid()
    print(f"Worker {name} (PID: {process_id}) starting")
    time.sleep(1)  # Simulate work
    print(f"Worker {name} (PID: {process_id}) finished")

if __name__ == '__main__':
    # Create multiple processes
    processes = []
    for i in range(3):
        # Create a process that runs worker_function
        p = multiprocessing.Process(
            target=worker_function, 
            args=(f'Process-{i}',)
        )
        processes.append(p)
        p.start()
    
    # Wait for all processes to complete
    for p in processes:
        p.join()
    
    print("All worker processes completed")

# CPU-bound task example - calculating prime numbers
print("\nCPU-bound task example - finding prime numbers:")

def is_prime(n):
    """Check if a number is prime (CPU-intensive)"""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def count_primes_in_range(start, end):
    """Count prime numbers in a given range"""
    count = sum(1 for i in range(start, end) if is_prime(i))
    return count

if __name__ == '__main__':
    # Define a large range to search for primes
    range_start = 1
    range_end = 100000
    
    # Split the work into chunks for parallel processing
    num_processes = multiprocessing.cpu_count()  # Use all available cores
    print(f"Using {num_processes} processes")
    
    chunk_size = (range_end - range_start) // num_processes
    chunks = [(range_start + i * chunk_size, 
              range_start + (i + 1) * chunk_size) 
             for i in range(num_processes)]
    
    # Measure time with multiprocessing
    start_time = time.time()
    
    # Process chunks in parallel
    with multiprocessing.Pool(processes=num_processes) as pool:
        results = pool.starmap(count_primes_in_range, chunks)
    
    total_primes = sum(results)
    mp_time = time.time() - start_time
    
    print(f"Found {total_primes} prime numbers in range {range_start}-{range_end}")
    print(f"Multiprocessing time: {mp_time:.2f} seconds")
    
    # Compare with single-process time (on a small subset for demonstration)
    small_end = 10000  # Smaller range for quick single-process demo
    start_time = time.time()
    single_result = count_primes_in_range(range_start, small_end)
    single_time = time.time() - start_time
    
    print(f"Single process found {single_result} primes in range {range_start}-{small_end}")
    print(f"Single process time: {single_time:.2f} seconds")

# Shared memory example
print("\nShared memory example:")

if __name__ == '__main__':
    # Create a shared value
    counter = multiprocessing.Value('i', 0)  # 'i' means signed integer
    
    def increment_counter(count, lock):
        for _ in range(count):
            with lock:  # Use lock to avoid race conditions
                counter.value += 1
    
    # Create a lock to protect the shared counter
    lock = multiprocessing.Lock()
    
    # Create processes that increment the same counter
    processes = []
    num_increments = 1000
    for _ in range(4):
        p = multiprocessing.Process(
            target=increment_counter, 
            args=(num_increments, lock)
        )
        processes.append(p)
        p.start()
    
    # Wait for all processes to complete
    for p in processes:
        p.join()
    
    print(f"Final counter value: {counter.value}")

# Data processing with multiprocessing in numpy (common in data science)
print("\nData processing with multiprocessing (numpy example):")

def process_chunk(chunk):
    """Process a chunk of data"""
    # Simulate complex data processing
    result = np.mean(chunk) * np.std(chunk)
    time.sleep(0.2)  # Simulate computation time
    return result

if __name__ == '__main__':
    # Create a large dataset
    data = np.random.rand(1000000)
    
    # Split data into chunks
    chunks = np.array_split(data, 4)
    
    # Process in parallel
    start_time = time.time()
    with multiprocessing.Pool(processes=4) as pool:
        results = pool.map(process_chunk, chunks)
    
    # Combine results
    combined_result = np.mean(results)
    mp_time = time.time() - start_time
    
    print(f"Parallel processing result: {combined_result:.6f}")
    print(f"Processing time: {mp_time:.2f} seconds")
    
    # Tips for effective multiprocessing
    print("\nTips for effective multiprocessing:")
    print("1. Always protect shared memory with locks")
    print("2. Minimize data transfer between processes")
    print("3. Use multiprocessing for CPU-bound tasks, not I/O-bound tasks")
    print("4. Use 'if __name__ == __main__' guard for Windows compatibility")
    print("5. Consider process pools for managing worker processes")
    print("6. Be aware of overhead - only use multiprocessing when tasks are substantial")

# Expected output will vary by system, but will include:
# - Multiple processes executing in parallel with different PIDs
# - Results from pool.map operations
# - Time comparisons showing parallel processing advantage
# - Shared memory counter results
# - Data processing results with numpy

Basic multiprocessing Pool example:


**Output:**
[1, 4, 9, 16]

**Real-life use case:** Image processing or data analysis on large datasets.

**Common mistakes:** Sharing state between processes (each process has its own memory space).

**Best practices:** Use multiprocessing for CPU-bound tasks and avoid sharing state.

## 3. Async/Await
**Definition:** Async/await enables asynchronous programming, useful for high-performance I/O-bound code.

**Syntax and Example:**

In [ ]:
import asyncio
import time
import aiohttp
import random

# Basic async example
async def say_hello():
    """A simple coroutine that waits and then prints a message"""
    await asyncio.sleep(1)  # Non-blocking sleep
    print('Hello after 1 second')
    return "Hello returned"  # Return value from coroutine

# Run a single coroutine
print("Basic async example:")
result = asyncio.run(say_hello())
print(f"Coroutine returned: {result}")

# Multiple coroutines running concurrently
print("\nMultiple coroutines running concurrently:")

async def process_item(item):
    """Process an item asynchronously"""
    # Simulate I/O operation with different durations
    delay = random.uniform(0.5, 1.5)
    print(f"Processing {item}, will take {delay:.2f} seconds")
    await asyncio.sleep(delay)  # Non-blocking sleep
    return f"Processed {item}"

async def main():
    """Run multiple coroutines concurrently"""
    start_time = time.time()
    
    # Create a list of coroutines to run
    tasks = [process_item(f"item-{i}") for i in range(5)]
    
    # Wait for all coroutines to complete and gather results
    results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    print(f"\nAll items processed in {end_time - start_time:.2f} seconds")
    print(f"Results: {results}")

# Run the main coroutine
asyncio.run(main())

# Comparing async with sequential execution
print("\nComparing async with sequential execution:")

async def fetch_data(session, url):
    """Fetch data from a URL asynchronously"""
    print(f"Fetching: {url}")
    try:
        # Asynchronous HTTP request
        async with session.get(url, timeout=5) as response:
            # Simulate varying response times
            delay = random.uniform(1, 3)
            await asyncio.sleep(delay)  # Simulate network delay
            return f"Result from {url}: {response.status}"
    except Exception as e:
        return f"Error fetching {url}: {str(e)}"

async def fetch_all_async():
    """Fetch multiple URLs asynchronously"""
    urls = [
        'https://example.com',
        'https://python.org',
        'https://github.com',
        'https://stackoverflow.com'
    ]
    
    start_time = time.time()
    
    # Create a client session
    async with aiohttp.ClientSession() as session:
        # Create tasks for each URL
        tasks = [fetch_data(session, url) for url in urls]
        # Wait for all tasks to complete
        results = await asyncio.gather(*tasks)
    
    end_time = time.time()
    print(f"\nAsync fetch completed in {end_time - start_time:.2f} seconds")
    for result in results:
        print(result)
    return end_time - start_time

def fetch_sequential():
    """Fetch multiple URLs sequentially (for comparison)"""
    urls = [
        'https://example.com',
        'https://python.org',
        'https://github.com',
        'https://stackoverflow.com'
    ]
    
    start_time = time.time()
    
    print("\nSequential fetch:")
    results = []
    for url in urls:
        print(f"Fetching: {url}")
        # Simulate a blocking request
        delay = random.uniform(1, 3)
        time.sleep(delay)  # Blocking sleep
        results.append(f"Result from {url}: 200")  # Simulated status
    
    end_time = time.time()
    print(f"\nSequential fetch completed in {end_time - start_time:.2f} seconds")
    for result in results:
        print(result)
    return end_time - start_time

# Run both implementations (comment out for notebook as it requires network)
# Note: actual network requests are simulated in this example
print("\nSimulated network requests:")
async_time = 0
seq_time = 0

try:
    # Will only work if aiohttp is installed
    import aiohttp
    print("Running async version (simulated):")
    async_time = asyncio.run(fetch_all_async())
except ModuleNotFoundError:
    print("aiohttp not installed, skipping async example")

# Run sequential version
print("Running sequential version (simulated):")
seq_time = fetch_sequential()

if async_time > 0 and seq_time > 0:
    print(f"\nSpeed comparison: Async is {seq_time / async_time:.1f}x faster")

# Async with timeout handling
print("\nAsync with timeout handling:")

async def slow_operation():
    """A slow operation that will be cancelled"""
    print("Slow operation started")
    await asyncio.sleep(10)  # Very long operation
    print("Slow operation completed")  # This won't be reached
    return "Slow result"

async def timeout_example():
    try:
        # Set a timeout of 2 seconds
        result = await asyncio.wait_for(slow_operation(), timeout=2.0)
        return result
    except asyncio.TimeoutError:
        print("Operation timed out!")
        return "Timeout result"

# Run with timeout
asyncio.run(timeout_example())

# Event loop explanation
print("\nAsync concepts:")
print("1. Event Loop: The core of async I/O, schedules and runs coroutines")
print("2. Coroutine: Function defined with 'async def', can be paused with 'await'")
print("3. Task: Wrapper around a coroutine to track its execution")
print("4. Future: Object representing a result that will be available later")
print("5. await: Pauses a coroutine until the awaited object (coroutine, task, future) completes")

print("\nBest practices for async/await:")
print("1. Use for I/O-bound tasks (network, file operations) not CPU-bound tasks")
print("2. Avoid blocking calls inside async functions")
print("3. Use asyncio.gather() to run multiple coroutines in parallel")
print("4. Handle exceptions properly in asynchronous code")
print("5. Use async libraries for I/O operations (aiohttp, asyncpg, etc.)")

# Expected output (with some variations due to random delays):
# Basic async example:
# Hello after 1 second
# Coroutine returned: Hello returned
#
# Multiple coroutines running concurrently:
# Processing item-0, will take 0.88 seconds
# Processing item-1, will take 1.23 seconds
# Processing item-2, will take 0.72 seconds
# Processing item-3, will take 1.15 seconds
# Processing item-4, will take 1.02 seconds
#
# All items processed in 1.23 seconds
# Results: ['Processed item-0', 'Processed item-1', 'Processed item-2', ...]
#
# Comparing async with sequential execution:
#
# Simulated network requests:
# Running async version (simulated):
# Fetching: https://example.com
# Fetching: https://python.org
# Fetching: https://github.com
# Fetching: https://stackoverflow.com
#
# Async fetch completed in 3.00 seconds
# Result from https://example.com: 200
# ...
#
# Running sequential version (simulated):
# Sequential fetch:
# Fetching: https://example.com
# Fetching: https://python.org
# Fetching: https://github.com
# Fetching: https://stackoverflow.com
#
# Sequential fetch completed in 9.50 seconds
# Result from https://example.com: 200
# ...
#
# Speed comparison: Async is 3.2x faster
#
# Async with timeout handling:
# Slow operation started
# Operation timed out!
#
# Async concepts:
# 1. Event Loop: The core of async I/O, schedules and runs coroutines
# ...
# Best practices for async/await:
# 1. Use for I/O-bound tasks (network, file operations) not CPU-bound tasks
# ...

**Output:**
Hello after 1 second

**Real-life use case:** Handling thousands of simultaneous web requests in a web server.

**Common mistakes:** Mixing blocking code with async code.

**Best practices:** Use async for I/O-bound, high-concurrency tasks.

## 4. Sockets
**Definition:** Sockets allow low-level network communication. Useful for building custom network protocols.

**Syntax and Example:** (Client)

In [ ]:
import socket
import threading
import time

# Basic socket client example
def simple_client():
    """A simple HTTP client using sockets"""
    # Create a socket object (AF_INET = IPv4, SOCK_STREAM = TCP)
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # Connect to a server (example.com on port 80)
        print("Connecting to example.com:80...")
        s.connect(('example.com', 80))
        
        # Send an HTTP GET request
        request = b'GET / HTTP/1.1\r\nHost: example.com\r\n\r\n'
        print(f"Sending request: {request}")
        s.sendall(request)
        
        # Receive the response (up to 4096 bytes)
        response = s.recv(4096)
        
        # Print the first part of the response
        print("\nResponse (first 300 bytes):")
        print(response[:300].decode('utf-8'))
        
    finally:
        # Always close the socket when done
        s.close()
        print("Socket closed")

# Uncomment to run (requires internet access)
# simple_client()

# Instead, let's see a local echo server and client example
print("Local echo server and client example:")

server_running = False
SERVER_HOST = '127.0.0.1'  # localhost
SERVER_PORT = 12345        # arbitrary non-privileged port

def echo_server():
    """A simple echo server that repeats back messages"""
    global server_running
    
    # Create server socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # Allow reuse of address (prevents "Address already in use" errors)
        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
        # Bind to host and port
        server_socket.bind((SERVER_HOST, SERVER_PORT))
        
        # Start listening for connections (5 is the backlog queue size)
        server_socket.listen(5)
        server_running = True
        print(f"Server listening on {SERVER_HOST}:{SERVER_PORT}")
        
        # Set a timeout so the server can be stopped
        server_socket.settimeout(10)  # 10 seconds timeout
        
        while server_running:
            try:
                # Accept incoming connection
                client_socket, client_address = server_socket.accept()
                print(f"Connection from {client_address}")
                
                # Handle client in a separate thread
                client_handler = threading.Thread(
                    target=handle_client,
                    args=(client_socket, client_address)
                )
                client_handler.daemon = True
                client_handler.start()
                
            except socket.timeout:
                # Just a timeout, continue the loop
                continue
            except Exception as e:
                print(f"Server error: {e}")
                break
    
    finally:
        # Clean up
        server_socket.close()
        server_running = False
        print("Server stopped")

def handle_client(client_socket, address):
    """Handle communication with a connected client"""
    try:
        # Receive data from client
        while True:
            data = client_socket.recv(1024)
            if not data:
                # No more data, client has disconnected
                break
            
            message = data.decode('utf-8')
            print(f"Received from {address}: {message.strip()}")
            
            # Echo back the data
            response = f"Echo: {message}"
            client_socket.sendall(response.encode('utf-8'))
    
    except Exception as e:
        print(f"Error handling client {address}: {e}")
    
    finally:
        # Close client socket
        client_socket.close()
        print(f"Connection with {address} closed")

def echo_client(messages):
    """A client that sends messages to the echo server"""
    # Create client socket
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        # Connect to server
        print(f"Connecting to {SERVER_HOST}:{SERVER_PORT}...")
        client_socket.connect((SERVER_HOST, SERVER_PORT))
        
        # Send messages
        for message in messages:
            print(f"Sending: {message}")
            client_socket.sendall(message.encode('utf-8'))
            
            # Wait for response
            response = client_socket.recv(1024).decode('utf-8')
            print(f"Received: {response}")
            
            # Short delay between messages
            time.sleep(0.5)
    
    except ConnectionRefusedError:
        print("Connection refused. Is the server running?")
    except Exception as e:
        print(f"Client error: {e}")
    
    finally:
        # Close socket
        client_socket.close()
        print("Client disconnected")

# Start the server in a background thread
server_thread = threading.Thread(target=echo_server)
server_thread.daemon = True
server_thread.start()

# Wait for server to start
time.sleep(1)

# Run client with messages
if server_running:
    messages = [
        "Hello, server!",
        "How are you?",
        "Goodbye!"
    ]
    echo_client(messages)

# Stop server
server_running = False
time.sleep(1)  # Give server time to clean up

# Socket concepts and best practices
print("\nSocket programming concepts:")
print("1. Socket Types:")
print("   - SOCK_STREAM (TCP): Reliable, ordered, connection-based")
print("   - SOCK_DGRAM (UDP): Unreliable, connectionless datagrams")

print("\n2. Socket Address Families:")
print("   - AF_INET: IPv4 addresses")
print("   - AF_INET6: IPv6 addresses")
print("   - AF_UNIX: Unix domain sockets (local inter-process communication)")

print("\n3. Common Socket Operations:")
print("   - socket(): Create a socket")
print("   - bind(): Bind socket to an address")
print("   - listen(): Set up socket for incoming connections")
print("   - accept(): Accept incoming connection")
print("   - connect(): Connect to a remote socket")
print("   - send()/recv(): Send/receive data")
print("   - close(): Close the socket")

print("\n4. Socket Options (with setsockopt):")
print("   - SO_REUSEADDR: Allow reusing local addresses")
print("   - SO_KEEPALIVE: Keep connection alive")
print("   - SO_RCVBUF/SO_SNDBUF: Receive/send buffer size")

print("\n5. Best Practices:")
print("   - Always close sockets when done")
print("   - Use try-finally blocks to ensure cleanup")
print("   - Handle partial sends/receives")
print("   - Set appropriate timeouts")
print("   - Consider using non-blocking sockets or async I/O for high performance")
print("   - Be aware of network errors and handle them gracefully")

# Note about socket vs requests
print("\nNote: While sockets provide low-level control, ")
print("for most HTTP needs, the 'requests' library is easier to use.")

# Expected output:
# Local echo server and client example:
# Server listening on 127.0.0.1:12345
# Connecting to 127.0.0.1:12345...
# Sending: Hello, server!
# Received: Echo: Hello, server!
# Sending: How are you?
# Received: Echo: How are you?
# Sending: Goodbye!
# Received: Echo: Goodbye!
# Client disconnected
# Server stopped
#
# Socket programming concepts:
# 1. Socket Types:
#    - SOCK_STREAM (TCP): Reliable, ordered, connection-based
#    - SOCK_DGRAM (UDP): Unreliable, connectionless datagrams
# ...etc...

**Output:**
HTTP/1.1 200 OK ... (HTML content)

**Real-life use case:** Building a chat application or custom server.

**Common mistakes:** Not closing sockets or handling partial data.

**Best practices:** Always close sockets and handle exceptions.

## 5. HTTP Requests (requests library)
**Definition:** The `requests` library is a user-friendly way to make HTTP requests in Python.

**Syntax and Example:**

In [ ]:
import requests
import json
from pprint import pprint
import time

# Basic GET request
print("Basic GET request:")
try:
    response = requests.get('https://api.github.com')
    print(f"Status code: {response.status_code}")
    print(f"Response headers: {dict(response.headers)[:3]}...")
    
    # Parse JSON response
    data = response.json()
    print("\nAPI endpoints available:")
    for endpoint, url in list(data.items())[:5]:
        print(f"  {endpoint}: {url}")
    print("  ...and more endpoints")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

# HTTP methods demonstration
print("\nHTTP methods with requests:")
crud_operations = [
    "GET: Retrieve data (requests.get())",
    "POST: Create new data (requests.post())",
    "PUT: Update existing data (requests.put())",
    "PATCH: Partially update data (requests.patch())",
    "DELETE: Remove data (requests.delete())"
]

for op in crud_operations:
    print(f"- {op}")

# Request with parameters
print("\nGET request with parameters:")
try:
    # Search GitHub repositories with query parameters
    params = {
        'q': 'python data science',
        'sort': 'stars',
        'order': 'desc',
        'per_page': 3
    }
    
    response = requests.get(
        'https://api.github.com/search/repositories', 
        params=params
    )
    
    print(f"Request URL: {response.url}")
    print(f"Status code: {response.status_code}")
    
    # Parse response (simulated)
    print("\nTop repositories for 'python data science' (simulated):")
    mock_results = [
        {"name": "pandas", "stars": 36500, "description": "Powerful data analysis toolkit"},
        {"name": "scikit-learn", "stars": 52000, "description": "Machine learning in Python"},
        {"name": "jupyter", "stars": 12000, "description": "Interactive computing"}
    ]
    
    for repo in mock_results:
        print(f"Repository: {repo['name']}")
        print(f"  Stars: {repo['stars']}")
        print(f"  Description: {repo['description']}")

except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

# POST request example
print("\nPOST request example:")

# Data to send
user_data = {
    "name": "John Doe",
    "job": "Data Scientist"
}

try:
    # Making a POST request to create a user (using JSONPlaceholder API)
    response = requests.post(
        'https://reqres.in/api/users',
        json=user_data  # Automatically serializes to JSON
    )
    
    print(f"Status code: {response.status_code}")
    
    # Parse the response
    data = response.json()
    print("\nCreated user (response):")
    pprint(data)
    
    # Show the added ID and creation timestamp
    print(f"\nUser ID: {data.get('id')}")
    print(f"Created at: {data.get('createdAt')}")
    
except requests.exceptions.RequestException as e:
    print(f"Request error: {e}")

# Working with headers and authentication
print("\nHeaders and authentication:")

def print_headers_example():
    # Custom headers
    headers = {
        'User-Agent': 'Python Requests Tutorial',
        'Accept': 'application/json',
        'X-Custom-Header': 'CustomValue'
    }
    
    # Basic authentication
    auth = ('username', 'password')  # Replace with actual credentials
    
    print("Example Headers:")
    for key, value in headers.items():
        print(f"  {key}: {value}")
    
    print("\nAuthentication Methods:")
    print("  - Basic Auth: requests.get(url, auth=('username', 'password'))")
    print("  - Bearer Token: headers = {'Authorization': 'Bearer YOUR_TOKEN'}")
    print("  - API Key: params = {'api_key': 'YOUR_API_KEY'}")
    print("  - OAuth: Use requests-oauthlib library")

print_headers_example()

# Handling errors and exceptions
print("\nHandling errors and exceptions:")

def demonstrate_error_handling():
    print("Common status codes:")
    status_codes = {
        200: "OK - Request successful",
        201: "Created - Resource created successfully",
        204: "No Content - Request successful, no content returned",
        400: "Bad Request - Invalid syntax or parameters",
        401: "Unauthorized - Authentication required",
        403: "Forbidden - Server refuses the request",
        404: "Not Found - Resource not found",
        500: "Internal Server Error - Server error occurred"
    }
    
    for code, desc in status_codes.items():
        print(f"  {code}: {desc}")
    
    print("\nException handling example:")
    print("""try:
    response = requests.get('https://api.example.com/data', timeout=5)
    response.raise_for_status()  # Raises an exception for 4XX/5XX responses
    data = response.json()
except requests.exceptions.HTTPError as e:
    print(f"HTTP error: {e}")
except requests.exceptions.ConnectionError as e:
    print(f"Connection error: {e}")
except requests.exceptions.Timeout as e:
    print(f"Timeout error: {e}")
except requests.exceptions.RequestException as e:
    print(f"General error: {e}")
""")

demonstrate_error_handling()

# Sessions and connection pooling
print("\nSessions and connection pooling:")

def session_example():
    # Create a session
    print("Using a session for multiple requests:")
    print("""session = requests.Session()

# Session will maintain cookies and connection pool
session.headers.update({'User-Agent': 'MyApp/1.0'})

# First request
response1 = session.get('https://api.example.com/data1')

# Second request (reuses connection and includes cookies from first request)
response2 = session.get('https://api.example.com/data2')

# Close the session when done
session.close()
""")
    
    print("\nBenefits of using Sessions:")
    print("  - Connection pooling (faster subsequent requests)")
    print("  - Automatic cookie persistence")
    print("  - Common configuration for multiple requests")

session_example()

# Best practices
print("\nRequests library best practices:")
best_practices = [
    "Always set timeouts to prevent hanging requests",
    "Use sessions for multiple requests to the same site",
    "Handle exceptions and check status codes",
    "Close response bodies and sessions when done",
    "Respect API rate limits",
    "Validate SSL certificates (don't use verify=False)",
    "Use connection pooling for performance",
    "Consider async alternatives (aiohttp, httpx) for high concurrency"
]

for i, practice in enumerate(best_practices, 1):
    print(f"{i}. {practice}")

# Data science specific examples
print("\nRequests in data science workflows:")
print("1. API data collection for analysis")
print("2. Downloading datasets from repositories")
print("3. Loading data into pandas from web services")
print("4. Web scraping for data gathering")
print("5. Publishing results to web services")

# Code example for downloading data with progress
print("\nExample: Downloading a dataset with progress tracking:")

def download_with_progress_example():
    print("""import requests
import os
import sys
from tqdm import tqdm

def download_file(url, filename):
    # Download a file with progress bar
    # Make request with stream=True to download in chunks
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    # Get total file size
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 KB
    
    # Create progress bar
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
    
    # Download and write file
    with open(filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    
    progress_bar.close()
    
    if total_size != 0 and progress_bar.n != total_size:
        print("Download incomplete")

# Example usage
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
download_file(url, "wine_dataset.csv")
""")

download_with_progress_example()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1561731844.py, line 219)